In [19]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

import seaborn as sns

sns.set(color_codes=True, style="white", context="talk", font_scale=1)
PALETTE = sns.color_palette("Set1")

from sklearn.metrics import pairwise_distances
from hyppo.independence import Dcorr
from scipy.stats import multiscale_graphcorr
from hyppo.ksample._utils import k_sample_transform
from hyppo.independence.dcorr import _center_distmat
from graspy.plot import heatmap

import sys

sys.path.append("../")
from src.tools.utils import get_files, get_latents

%load_ext autoreload
%autoreload 2
%load_ext nb_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [2]:
sns.set(color_codes=True, style="white", context="poster", font_scale=2)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[3:])

<IPython.core.display.Javascript object>

In [3]:
base_dir = Path("/mnt/ssd3/ronan/data/gcca_05-26-10:39_min_rank-ZG3/")
# base_dir = Path("/mnt/ssd3/ronan/data/gcca_05-17-18:27_max_rank-ZG2/")
groups, labels = get_latents(base_dir, flag="_gcca")

<IPython.core.display.Javascript object>

In [4]:
labels

[['e', 'restingstate'],
 ['e', 'openmonitoring'],
 ['e', 'compassion'],
 ['n', 'restingstate'],
 ['n', 'openmonitoring'],
 ['n', 'compassion']]

<IPython.core.display.Javascript object>

In [31]:
ways = [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2]]
X, Y = k_sample_transform(groups, ways=ways)
X = X[:, :, :3]

<IPython.core.display.Javascript object>

## Effects to test
Each gradient
- Expert vs Novice (global)
- Expert vs Novice (per state)
- Expert vs Expert (state-vs-other)

In [48]:
def get_sum(c_mat, Y, trait, states, normalized=True):
    """
    
    """
    trait_idx = np.where(Y[:, trait] == 1)[0]
    states_idxs = []
    for state in states:
        states_idxs.append(set(np.where(Y[:, state + 2] == 1)[0]))
    idx = list(set.union(*states_idxs) & set(trait_idx))

    return np.sum(c_mat[np.ix_(idx, idx)])

<IPython.core.display.Javascript object>

In [50]:
effects = []
interactions = [
    "Expert - Novice",
    "Expert (resting state) - Novice (resting state)",
    "Expert (open monitoring) - Novice (open monitoring)",
    "Expert (compassion) - Novice (compassion)",
    "Expert (Meditating) - Novice (Resting)",
    "Expert (resting) - Expert (other)",
    "Expert (open monitoring) - Expert (other)",
    "Expert (compassion) - Expert (other)",
]
for gradient in [0, 1, 2]:
    effs = []
    X_dists = pairwise_distances(X[:, :, gradient], metric="euclidean")
    Y_dists = pairwise_distances(Y[:, :2], metric="sqeuclidean") / 2

    X_centered_dists = _center_distmat(X_dists, bias=False)
    Y_centered_dists = _center_distmat(Y_dists, bias=False)

    c_mat = X_centered_dists * Y_centered_dists

    ## e vs n global
    effs.append((get_sum(c_mat, Y, 0, [0, 1, 2]) - get_sum(c_mat, Y, 1, [0, 1, 2])) / 9)
    ## e vs n statewise
    for state in range(3):
        effs.append((get_sum(c_mat, Y, 0, [state]) - get_sum(c_mat, Y, 1, [state])))
    ## e meditating vs n resting
    effs.append((get_sum(c_mat, Y, 0, [1, 2]) / 4 - get_sum(c_mat, Y, 1, [0])))
    ## e vs e state-vs-other
    for state in range(3):
        other = list(set(range(3)) - set([state]))
        effs.append((get_sum(c_mat, Y, 0, [state]) - get_sum(c_mat, Y, 0, other) / 4))

    effects.append(effs)

<IPython.core.display.Javascript object>

In [74]:
df = pd.DataFrame(
    np.vstack((interactions, *np.asarray(effects).astype(int))).T,
    columns=["Effect", "Gradient 1", "Gradient 2", "Gradient 3"],
).set_index("Effect")

<IPython.core.display.Javascript object>

In [75]:
df

,Gradient 1,Gradient 2,Gradient 3
Effect,,,
Expert - Novice,154,139,162
Expert (resting state) - Novice (resting state),26,167,-268
Expert (open monitoring) - Novice (open monitoring),-253,-58,239
Expert (compassion) - Novice (compassion),143,-183,333
Expert (Meditating) - Novice (Resting),320,324,319
Expert (resting) - Expert (other),-293,-157,-587
Expert (open monitoring) - Expert (other),-314,-114,-24
Expert (compassion) - Expert (other),-169,-402,388


<IPython.core.display.Javascript object>

In [79]:
df.to_csv("../reports/ohbm/effect_sizes.csv")

<IPython.core.display.Javascript object>

## Standardized by Permutation